In [ ]:
import pennylane as qml


def prepare_state(phi, theta, omega, wire=None):
    """Use this subroutine to prepare a quantum state according to three angular
    parameters."""
    qml.Hadamard(wires=wire)
    qml.Rot(phi, theta, omega, wires=wire)


def entanglement_factory(wires=None):
    """A subroutine to create a two-qubit entangled state.

    Args:
        (qml.Wires or List[int] or List[str]): A set of two wires to create the
            entangled state between.
    """
    # YOUR CODE HERE
    qml.Hadamard(wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])


def entanglement_swap(wires=None):
    """A subroutine to perform a (deferred) Bell measurement to swap entanglement
    between qubits in a network.

    Args:
        wires (qml.Wires or List[int] or List[str]): A set of three wires.
    """
    # YOUR CODE HERE
    qml.adjoint(entanglement_factory)(wires=[wires[0], wires[1]])

    m1 = qml.measure(wires=wires[0])
    m2 = qml.measure(wires=wires[1])

    qml.cond(m2, qml.PauliX)(wires=wires[2]) # conditioned on bit m2, apply PauliX
    qml.cond(m1, qml.PauliZ)(wires=wires[2]) # conditioned on bit m1, apply PauliZ


def quantum_network(wires):
    """Implement a quantum network.

    The network circuit should be sufficiently general as to work for an arbitrarily
    specified list of wires. It should perform the following:
     - use the prepare_state function to prepare a state with the given parameters
       on the first wire
     - set up the pairs of entangled states on the path from Alice to Bob
     - use entanglement swapping to teleport Alice's state through the network to Bob
     - return the measurement outcome probabilities of Bob's qubit

    Args:
        wires (qml.Wires or List[int] or List[str]): The list of wires on the device.

    Returns:
        qml.QNode: the QNode that implements the communication protocol. This QNode
        should return the measurement outcome probabilities of the final qubit.
    """
    dev = qml.device("default.qubit", wires=wires)
    
    @qml.qnode(dev)
    def send_quantum_information(phi, theta, omega):
        """Send the quantum information through the network.

        Args:
            phi, theta, omega (float): Parameters for an initial qml.Rot gate that
                will prepare an arbitrary quantum state.

        Returns:
            array[float]: A length-2 array with the measurement outcome
            probabilities of Bob's qubit.
        """
        # YOUR CODE HERE
        n = len(wires)
        prepare_state(phi, theta, omega, wires[0])
        for i in range(0, n - 2, 2):
            entanglement_factory(wires=[wires[i + 1], wires[i + 2]])
            entanglement_swap(wires=[wires[i], wires[i + 1], wires[i + 2]])
        return qml.probs(wires=wires[-1])

    return send_quantum_information